In [ ]:
using DynamicPolynomials
using SumOfSquares
using CSDP
using JuMP
using Plots
using LinearAlgebra
using DifferentialEquations
using PyCall
using MosekTools

$$x_1' = x_2, x_2' = -x_1 + u $$ 

# 0. Init system variable and parameters

In [ ]:
#init state and input variables
@polyvar x[1:2]
@polyvar u[1:1]

#define system dynamics
vectorField = [ x[2], -x[1] + u[1] ] 

#define unsafe set (obstacle)
g = 0.1^2 - x[1]^2 - x[2]^2

#state and input bounds
bounds = [[-10., 10.], [-10., 10.] ]
u_bounds = [[-1.,1.]]

In [ ]:
function prepare_domain(x::Vector{<:Variable}, bounds::Vector{Vector{Float64}})
   poly_list = [ 
        @set(x[i] - l >= 0) ∩ @set(u - x[i] >= 0 ) ∩ @set((x[i] -l)*(u-x[i]) >= 0)
        for (i, (l, u)) in enumerate(bounds)
        ] 
   poly_list
end

In [ ]:
#instantiate parameters
λ = 1
ϵ = 1
max_degree=4
U = [[-1.0],[1.0]]

In [ ]:
# generate test points
function get_random(limits::Vector{Vector{Float64}}, g::Polynomial)
    function get_random_scalar(lb, ub )
        lb + rand()*(ub - lb) 
    end
    while (true)
        pt = [get_random_scalar(l[1], l[2]) for l in limits]
        if g(pt[1], pt[2]) >= 0
            continue
        else
            return pt
        end
    end
end

test_pts = [ get_random(bounds, g) for _ in 1:25];

# 1. Computing initial set of barriers for each input U_i

In [ ]:
# function to generate initial barriers for each input
using Clarabel

function generate_barrier(x, u, bounds, g, vectorField, U, test_pts; max_degree=4,ϵ = 0.25, λ = 0.1, γ = 10.)
    solver = optimizer_with_attributes(CSDP.Optimizer)
#     solver = optimizer_with_attributes(MosekTools.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, -γ <= c[1:N] <= γ)
    
    B = polynomial(c[big"1":end], monos) 
    #negative inside the domain
    @constraint(model, cons1, B <= -ϵ, domain=dom ∩ @set(g >= 0))
    B_dot = dot(differentiate(B,x), vectorField)
    B_dot_with_u = subs(B_dot, u => U)
    @constraint(model, cons2, B_dot_with_u >= λ * B, domain=dom)
    
    set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    objective_fn = sum([B(pt...) for pt in test_pts])
    @objective(model, Max, objective_fn) # keep as many points outside the barrier as you can
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    lm = [lagrangian_multipliers(cons1)]
    push!(lm, lagrangian_multipliers(cons2))
    value(B), value(B_dot), lm
end

In [ ]:
# function to comupte transit time for each barrier 

function refine_barrier_dn(x, u, bounds, u_bounds, g, vectorField, B, B_dot; ϵ = 0.25, κ = .1)
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom_list_u = prepare_domain(u, u_bounds)
    dom_list = append!(dom_list, dom_list_u)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, η)
    @variable(model, δ >=0)
    dom3 = dom ∩ @set(B >= 0) ∩ @set(B <= κ)
    @constraint(model, -η*B - δ <= B_dot, domain=dom3)
    @constraint(model, η*κ + δ >= 0)
    @objective(model, Min, δ)
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    println("η = $(JuMP.value(η))")
    println("δ = $(JuMP.value(δ))")
    τ =  (κ)/max(JuMP.value(δ),JuMP.value(η) *  κ + JuMP.value(δ))
    println("τd = $(JuMP.value(τ))")
    return τ
end

In [ ]:
elapsed1 = @elapsed begin
B_0, B_0d, lm0 = generate_barrier(x, u, bounds, g, vectorField, U[1], test_pts; max_degree = 2)
end
display(B_0)
display(B_0d)
t_0d = refine_barrier_dn(x, u, bounds, u_bounds, g, vectorField, B_0,  B_0d)

if (!ismissing(B_0))
    test_pts = filter!(pt -> B_0(pt...) <= 0., test_pts)
end

In [ ]:
l = lm0[1][1].basis.monomials'*lm0[1][1].Q*lm0[1][1].basis.monomials

In [ ]:
elapsed2 = @elapsed begin
B_1, B_1d, lm1 = generate_barrier(x, u, bounds, g, vectorField, U[2], test_pts; max_degree=2)
end
display(B_1)
display(B_1d)
t_1d = refine_barrier_dn(x, u, bounds, u_bounds, g, vectorField, B_1,  B_1d)

if (!ismissing(B_1))
    test_pts = filter!(pt -> B_1(pt...) <= 0., test_pts)
end

# 2. Computing Successive Barriers

In [ ]:
# functions to compute successive barriers

In [ ]:
function generate_successive_barrier(x::Vector{<:Variable}, u::Vector{<:Variable}, 
        bounds::Vector{Vector{Float64}}, g::Polynomial, 
        vectorField::Vector{<:Polynomial}, U::Vector{Float64}, 
        test_pts::Vector{Vector{Float64}}, ancestors::Vector{<:Polynomial}, 
        pt_to_eliminate::Vector{Float64}; 
        max_degree=4,ϵ = 1, λ = 1, γ = 10., κ = .1)
    solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")

    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, -γ <= c[1:N] <= γ)
    
    B = polynomial(c[1:end], monos) 
    # negative inside the obstacle
    @constraint(model, cons1, B <= -ϵ, domain=dom ∩ @set(g >= 0) )
    
    # dynamics constraints
    B_dot = dot(differentiate(B,x), vectorField)
    B_dot_with_u = subs(B_dot, u => U)
    if size(ancestors)[1] >= 1
        new_domain = dom ∩ (reduce(∩, [@set(B <= 0) for B in ancestors]))
    else
        new_domain = dom 
    end
    @constraint(model, cons2, B_dot_with_u >= λ * B, domain=new_domain)
    
    # eliminate the point we would like to eliminate
    cons3 = @constraint(model, B(pt_to_eliminate...) >= ϵ)
    set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    
    # maximize the sum of values for all test points
    objective_fn = sum([B(pt...) for pt in test_pts])
    @objective(model, Max, objective_fn) # keep as many points outside the barrier as you can
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    lm = [lagrangian_multipliers(cons1)]
    push!(lm, lagrangian_multipliers(cons2))
    value(B), value(B_dot), lm, cons3
end

In [ ]:
function refine_barrier_succ_dn(x, u, bounds, u_bounds, g, vectorField, B, B_dot, ancestors; ϵ = 0.25, κ = .1)
#     solver = optimizer_with_attributes(CSDP.Optimizer)
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom_list_u = prepare_domain(u, u_bounds)
    dom_list = append!(dom_list, dom_list_u)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
    #println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, η)
    @variable(model, δ>=0)

    if size(ancestors)[1] >= 1
        new_domain = dom ∩ (reduce(∩, [@set(b <= 0) for b in ancestors]))
    else
        new_domain = dom 
    end
    
    dom3 = new_domain ∩ @set(B >= 0) ∩ @set(B <= κ)
    @constraint(model, -η*B - δ <= B_dot, domain=dom3)
    @constraint(model, η*κ + δ >= 0)

    #set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    @objective(model, Min, δ)
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    println("η = $(value(η))")
    println("δ = $(value(δ))")
    τ =  (κ)/max(value(δ),value(η) *  κ + value(δ))
    println("τd = $(value(τ))")
    return τ
end

In [ ]:
function compute_next_level_barriers(x::Vector{<:Variable}, u::Vector{<:Variable}, bounds::Vector{Vector{Float64}},
                                    u_bounds::Vector{Vector{Float64}},
                                    g::Polynomial, vectorField::Vector{<:Polynomial}, 
                                    U::Vector{Vector{Float64}}, test_pts::Vector{Vector{Float64}}, 
                                    ancestors::Vector{<:Polynomial}) 
    eliminated = []
    second_level_barriers=[]
    for (j,pt) in enumerate(test_pts) 
        if j in eliminated
            # ignore the ones already eliminated
            continue
        end
        # go through each control 
        for u_val in U
            # generate a barrier 
            B = generate_successive_barrier(x, u, bounds, g, vectorField, u_val, test_pts, ancestors, pt)
            # check if we found something
            if !ismissing(B)
                println("Bingo: Found $B")
                useful = false # check if it is actually useful
                for (k,pt_new) in enumerate(test_pts)
                    if k in eliminated
                        continue
                    end
                   if (B[1](pt_new...) >= 0.)
                    push!(eliminated, k)   
                    println("\t eliminated $pt_new")
                    useful=true
                   end
                end
                if (useful)
                    println("Num remaining = $(size(test_pts)[1] - size(eliminated)[1])")
                    td = refine_barrier_succ_dn(x, u, bounds, u_bounds, g, vectorField, B[1],  B[2], ancestors)         
                    push!(second_level_barriers, (B[1], u_val, td, B[3], B[4]))
                    break
                end
            end
        end
    end
    new_test_pts = [pt for (j, pt) in enumerate(test_pts) if !(j in eliminated)]
    return (second_level_barriers, new_test_pts)
end

In [ ]:
zero_level_barriers::Vector{Polynomial} =[B_0, B_1]
print(size(zero_level_barriers))
s_elapsed = @elapsed begin
(first_level_barriers, test_pts_1) = compute_next_level_barriers(x, u, bounds, u_bounds, g, vectorField, U, test_pts, zero_level_barriers)
end

In [ ]:
test_pts_1 = [ get_random(bounds, g) for _ in 1:25];

In [ ]:
all_barriers1::Vector{Polynomial} = []
append!(all_barriers1, zero_level_barriers)
append!(all_barriers1, [B for (B, _) in first_level_barriers])
(second_level_barriers, test_pts_2) = compute_next_level_barriers(x, u, bounds, u_bounds, g, vectorField, U, test_pts_1, all_barriers1)

# 3. Plots

In [ ]:
# plotting functions

function circleShape(x,y, r)
    θ = LinRange(0, 2*π, 500)
    x .+ r*cos.(θ), y .+ r*sin.(θ)
end

function plot_ci_region2lev(limits::Tuple{Float64,Float64}, lev1_barriers::Vector{<:Polynomial}, lev2_barriers::Vector{<:Polynomial}; δ = 0.1, theta_val=0.0, filestem="ics")
    plot(xlims=limits, ylims=limits)
    rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])
    for x in limits[1]:δ:limits[2]
        for y in limits[1]:δ:limits[2]
            if (any([ B(x, y) > 0. for B in lev1_barriers]))
                plot!(rectangle(δ, δ, x-δ, y-δ), label=false, fill=:seagreen1, opacity=0.5,linecolor=:seagreen1)
            else
                if (any([ B(x, y) > 0. for B in lev2_barriers]))
                    plot!(rectangle(δ, δ, x-δ, y-δ), label=false, fill=:limegreen, linecolor=:limegreen)
                end 
            end
        end
    end
    plot!(circleShape(0,0,0.2), seriestype =[:shape], lw=0.5, c=:black, linecolor=:black, legend=false, aspectratio=1  )
    plot!(xlims=limits, ylims=limits)
    filename="figures/$filestem-theta-$(round(theta_val; digits=2)).png"
    savefig(filename)
    plot!(xlims=limits, ylims=limits)
end


In [ ]:
# improvement in the control invariant region at level 1
all_barriers = [B for (B,_) in first_level_barriers]
append!(all_barriers, [B_0, B_1]);
plot_ci_region2lev((-10., 10.), [B_0, B_1], all_barriers; δ=0.25, theta_val=-3.1415/2, filestem="Figure7a-poly1-lev1")

In [ ]:
# improvement in the control invariant region at level 2
lev2_barriers::Vector{Polynomial} = [B for (B,_) in second_level_barriers]
plot_ci_region2lev((-10., 10.0), all_barriers, lev2_barriers; theta_val=3.14, δ=0.25, filestem="Figure7a-poly1-lev2")

In [ ]:
# minimum transit time over all barriers
t0_min = minimum([t_0d, t_1d]);
t1=[]
for i in first_level_barriers
    if (!ismissing(i[3]))
        t1 = append!(t1,i[3])
    end
end
t1 = filter(x -> x > 0, t1)
t1_min = minimum(t1);
t2=[]
for i in second_level_barriers
    if (!ismissing(i[3]))
        t2 = append!(t2,i[3])
    end
end
t2 = filter(x -> x > 0, t2);
t2_min = minimum(t2);

println("τ_min = $(minimum([t0_min, t1_min, t2_min]))")

# 4. Benchmarks

In [ ]:
# generating data for Table 1 in the paper (row for current system)

In [ ]:
println("Time taken for B1: $(elapsed1+elapsed2)")
println("# barriers B1: $(size(zero_level_barriers))")
println("Time taken for B2: $(s_elapsed)")
println("# barriers B2: $(size(first_level_barriers))")

In [ ]:
# generating test points with fixed seed for evaluations (Table 2 in paper)

function generate_random_points_fixed_seed(limits::Vector{Vector{Float64}}, g::Polynomial)
     function get_random_scalar(lb, ub )
        Random.seed!(1234)
        lb + rand()*(ub - lb) 
     end
     while (true)
         pt = [get_random_scalar(l[1], l[2]) for l in limits]
         if g(pt[1], pt[2]) >= 0
             continue
         else
             return pt
         end
     end
end
test_pts_fixed_seed = [ get_random(bounds, g) for _ in 1:1000];

In [ ]:
# functions for FOSSIL comparison (Table 2 in paper)

py"""
import math
def eval_fossil_barrier(x0,x1):
    barrier = (1.5352206230163574 - 1.0230000019073486 * pow((-1.9589999914169312 + (-3.005000114440918 / (1 + math.exp((4.5869998931884766 - 0.14499999582767487 * x0 + 2.1189999580383301 * x1)))) + (-1.5240000486373901 / (1 + math.exp((1.4500000476837158 + 2.2709999084472656 * x0 - 0.035999998450279236 * x1)))) + (-0.93999999761581421 / (1 + math.exp((4.7290000915527344 - 2.1909999847412109 * x0 + 1.8890000581741333 * x1)))) + (0.7850000262260437 / (1 + math.exp((0.44100001454353333 + 0.86799997091293335 * x0 - 2.3900001049041748 * x1)))) + (1.0590000152587891 / (1 + math.exp((2.2880001068115234 - 2.7179999351501465 * x0 + 0.27500000596046448 * x1)))) + (1.0839999914169312 / (1 + math.exp((1.6000000238418579 - 3.7170000076293945 * x0 - 1.3589999675750732 * x1)))) + (1.1330000162124634 / (1 + math.exp((0.18500000238418579 + 0.59799998998641968 * x0 - 2.3910000324249268 * x1)))) + (1.156000018119812 / (1 + math.exp((-1.5820000171661377 - 0.32499998807907104 * x0 - 2.9619998931884766 * x1)))) + (1.2070000171661377 / (1 + math.exp((0.89200001955032349 + 2.6979999542236328 * x0 - 2.875999927520752 * x1)))) + (1.2920000553131104 / (1 + math.exp((5.0430002212524414 - 3.7460000514984131 * x0 - 0.57999998331069946 * x1))))), 2) - 0.5910000205039978 * pow((-1.9170000553131104 + (-2.1970000267028809 / (1 + math.exp((4.5869998931884766 - 0.14499999582767487 * x0 + 2.1189999580383301 * x1)))) + (-0.92299997806549072 / (1 + math.exp((4.7290000915527344 - 2.1909999847412109 * x0 + 1.8890000581741333 * x1)))) + (-0.51099997758865356 / (1 + math.exp((1.4500000476837158 + 2.2709999084472656 * x0 - 0.035999998450279236 * x1)))) + (0.050999999046325684 / (1 + math.exp((0.44100001454353333 + 0.86799997091293335 * x0 - 2.3900001049041748 * x1)))) + (0.52600002288818359 / (1 + math.exp((2.2880001068115234 - 2.7179999351501465 * x0 + 0.27500000596046448 * x1)))) + (0.81599998474121094 / (1 + math.exp((0.89200001955032349 + 2.6979999542236328 * x0 - 2.875999927520752 * x1)))) + (0.86100000143051147 / (1 + math.exp((0.18500000238418579 + 0.59799998998641968 * x0 - 2.3910000324249268 * x1)))) + (0.90600001811981201 / (1 + math.exp((1.6000000238418579 - 3.7170000076293945 * x0 - 1.3589999675750732 * x1)))) + (0.94300001859664917 / (1 + math.exp((-1.5820000171661377 - 0.32499998807907104 * x0 - 2.9619998931884766 * x1)))) + (1.0199999809265137 / (1 + math.exp((5.0430002212524414 - 3.7460000514984131 * x0 - 0.57999998331069946 * x1))))), 2) - 0.30700001120567322 * pow((-0.92299997806549072 + (-1.4620000123977661 / (1 + math.exp((4.7290000915527344 - 2.1909999847412109 * x0 + 1.8890000581741333 * x1)))) + (-1.0989999771118164 / (1 + math.exp((1.4500000476837158 + 2.2709999084472656 * x0 - 0.035999998450279236 * x1)))) + (-0.58399999141693115 / (1 + math.exp((4.5869998931884766 - 0.14499999582767487 * x0 + 2.1189999580383301 * x1)))) + (-0.29699999094009399 / (1 + math.exp((0.18500000238418579 + 0.59799998998641968 * x0 - 2.3910000324249268 * x1)))) + (0.05299999937415123 / (1 + math.exp((2.2880001068115234 - 2.7179999351501465 * x0 + 0.27500000596046448 * x1)))) + (0.20000000298023224 / (1 + math.exp((0.44100001454353333 + 0.86799997091293335 * x0 - 2.3900001049041748 * x1)))) + (0.2199999988079071 / (1 + math.exp((1.6000000238418579 - 3.7170000076293945 * x0 - 1.3589999675750732 * x1)))) + (0.61400002241134644 / (1 + math.exp((0.89200001955032349 + 2.6979999542236328 * x0 - 2.875999927520752 * x1)))) + (0.61500000953674316 / (1 + math.exp((-1.5820000171661377 - 0.32499998807907104 * x0 - 2.9619998931884766 * x1)))) + (1.2120000123977661 / (1 + math.exp((5.0430002212524414 - 3.7460000514984131 * x0 - 0.57999998331069946 * x1))))), 2) + 1.2150000333786011 * pow((-0.58799999952316284 + (-4.5580000877380371 / (1 + math.exp((1.6000000238418579 - 3.7170000076293945 * x0 - 1.3589999675750732 * x1)))) + (-2.7660000324249268 / (1 + math.exp((2.2880001068115234 - 2.7179999351501465 * x0 + 0.27500000596046448 * x1)))) + (-2.687999963760376 / (1 + math.exp((4.5869998931884766 - 0.14499999582767487 * x0 + 2.1189999580383301 * x1)))) + (-1.340999960899353 / (1 + math.exp((5.0430002212524414 - 3.7460000514984131 * x0 - 0.57999998331069946 * x1)))) + (-1.1579999923706055 / (1 + math.exp((4.7290000915527344 - 2.1909999847412109 * x0 + 1.8890000581741333 * x1)))) + (-0.17700000107288361 / (1 + math.exp((0.89200001955032349 + 2.6979999542236328 * x0 - 2.875999927520752 * x1)))) + (0.48600000143051147 / (1 + math.exp((0.18500000238418579 + 0.59799998998641968 * x0 - 2.3910000324249268 * x1)))) + (0.80800002813339233 / (1 + math.exp((-1.5820000171661377 - 0.32499998807907104 * x0 - 2.9619998931884766 * x1)))) + (1.6000000238418579 / (1 + math.exp((0.44100001454353333 + 0.86799997091293335 * x0 - 2.3900001049041748 * x1)))) + (2.937999963760376 / (1 + math.exp((1.4500000476837158 + 2.2709999084472656 * x0 - 0.035999998450279236 * x1))))), 2) - 0.79900002479553223 * pow((-0.48100000619888306 + (-1.8619999885559082 / (1 + math.exp((1.4500000476837158 + 2.2709999084472656 * x0 - 0.035999998450279236 * x1)))) + (-1.4819999933242798 / (1 + math.exp((4.5869998931884766 - 0.14499999582767487 * x0 + 2.1189999580383301 * x1)))) + (-1.4409999847412109 / (1 + math.exp((4.7290000915527344 - 2.1909999847412109 * x0 + 1.8890000581741333 * x1)))) + (0.25699999928474426 / (1 + math.exp((0.89200001955032349 + 2.6979999542236328 * x0 - 2.875999927520752 * x1)))) + (0.44299998879432678 / (1 + math.exp((-1.5820000171661377 - 0.32499998807907104 * x0 - 2.9619998931884766 * x1)))) + (0.69099998474121094 / (1 + math.exp((0.18500000238418579 + 0.59799998998641968 * x0 - 2.3910000324249268 * x1)))) + (1.2660000324249268 / (1 + math.exp((1.6000000238418579 - 3.7170000076293945 * x0 - 1.3589999675750732 * x1)))) + (1.2829999923706055 / (1 + math.exp((2.2880001068115234 - 2.7179999351501465 * x0 + 0.27500000596046448 * x1)))) + (1.6579999923706055 / (1 + math.exp((5.0430002212524414 - 3.7460000514984131 * x0 - 0.57999998331069946 * x1)))) + (1.7849999666213989 / (1 + math.exp((0.44100001454353333 + 0.86799997091293335 * x0 - 2.3900001049041748 * x1))))), 2) - 0.019999999552965164 * pow((-0.094999998807907104 + (-0.72399997711181641 / (1 + math.exp((1.4500000476837158 + 2.2709999084472656 * x0 - 0.035999998450279236 * x1)))) + (-0.041000001132488251 / (1 + math.exp((4.5869998931884766 - 0.14499999582767487 * x0 + 2.1189999580383301 * x1)))) + (0.17000000178813934 / (1 + math.exp((0.18500000238418579 + 0.59799998998641968 * x0 - 2.3910000324249268 * x1)))) + (0.24199999868869781 / (1 + math.exp((-1.5820000171661377 - 0.32499998807907104 * x0 - 2.9619998931884766 * x1)))) + (0.41499999165534973 / (1 + math.exp((4.7290000915527344 - 2.1909999847412109 * x0 + 1.8890000581741333 * x1)))) + (0.55299997329711914 / (1 + math.exp((5.0430002212524414 - 3.7460000514984131 * x0 - 0.57999998331069946 * x1)))) + (0.82300001382827759 / (1 + math.exp((0.44100001454353333 + 0.86799997091293335 * x0 - 2.3900001049041748 * x1)))) + (1.1150000095367432 / (1 + math.exp((0.89200001955032349 + 2.6979999542236328 * x0 - 2.875999927520752 * x1)))) + (1.2419999837875366 / (1 + math.exp((2.2880001068115234 - 2.7179999351501465 * x0 + 0.27500000596046448 * x1)))) + (1.7000000476837158 / (1 + math.exp((1.6000000238418579 - 3.7170000076293945 * x0 - 1.3589999675750732 * x1))))), 2) - 0.23399999737739563 * pow((0.057000000029802322 + (-1.4589999914169312 / (1 + math.exp((4.7290000915527344 - 2.1909999847412109 * x0 + 1.8890000581741333 * x1)))) + (-1.034000039100647 / (1 + math.exp((5.0430002212524414 - 3.7460000514984131 * x0 - 0.57999998331069946 * x1)))) + (-0.71899998188018799 / (1 + math.exp((-1.5820000171661377 - 0.32499998807907104 * x0 - 2.9619998931884766 * x1)))) + (-0.66100001335144043 / (1 + math.exp((1.4500000476837158 + 2.2709999084472656 * x0 - 0.035999998450279236 * x1)))) + (-0.38199999928474426 / (1 + math.exp((0.18500000238418579 + 0.59799998998641968 * x0 - 2.3910000324249268 * x1)))) + (-0.29100000858306885 / (1 + math.exp((2.2880001068115234 - 2.7179999351501465 * x0 + 0.27500000596046448 * x1)))) + (-0.23999999463558197 / (1 + math.exp((1.6000000238418579 - 3.7170000076293945 * x0 - 1.3589999675750732 * x1)))) + (-0.11699999868869781 / (1 + math.exp((4.5869998931884766 - 0.14499999582767487 * x0 + 2.1189999580383301 * x1)))) + (0.51499998569488525 / (1 + math.exp((0.89200001955032349 + 2.6979999542236328 * x0 - 2.875999927520752 * x1)))) + (0.76999998092651367 / (1 + math.exp((0.44100001454353333 + 0.86799997091293335 * x0 - 2.3900001049041748 * x1))))), 2) + 0.1809999942779541 * pow((0.55000001192092896 + (-1.3609999418258667 / (1 + math.exp((0.18500000238418579 + 0.59799998998641968 * x0 - 2.3910000324249268 * x1)))) + (-0.92100000381469727 / (1 + math.exp((5.0430002212524414 - 3.7460000514984131 * x0 - 0.57999998331069946 * x1)))) + (-0.3919999897480011 / (1 + math.exp((4.5869998931884766 - 0.14499999582767487 * x0 + 2.1189999580383301 * x1)))) + (-0.016000000759959221 / (1 + math.exp((4.7290000915527344 - 2.1909999847412109 * x0 + 1.8890000581741333 * x1)))) + (0.016000000759959221 / (1 + math.exp((1.6000000238418579 - 3.7170000076293945 * x0 - 1.3589999675750732 * x1)))) + (0.12399999797344208 / (1 + math.exp((0.44100001454353333 + 0.86799997091293335 * x0 - 2.3900001049041748 * x1)))) + (0.20299999415874481 / (1 + math.exp((2.2880001068115234 - 2.7179999351501465 * x0 + 0.27500000596046448 * x1)))) + (0.48199999332427979 / (1 + math.exp((1.4500000476837158 + 2.2709999084472656 * x0 - 0.035999998450279236 * x1)))) + (1.7430000305175781 / (1 + math.exp((-1.5820000171661377 - 0.32499998807907104 * x0 - 2.9619998931884766 * x1)))) + (3.0130000114440918 / (1 + math.exp((0.89200001955032349 + 2.6979999542236328 * x0 - 2.875999927520752 * x1))))), 2) - 0.48199999332427979 * pow((1.1950000524520874 + (-1.1959999799728394 / (1 + math.exp((0.18500000238418579 + 0.59799998998641968 * x0 - 2.3910000324249268 * x1)))) + (-1.0210000276565552 / (1 + math.exp((-1.5820000171661377 - 0.32499998807907104 * x0 - 2.9619998931884766 * x1)))) + (-0.5350000262260437 / (1 + math.exp((0.44100001454353333 + 0.86799997091293335 * x0 - 2.3900001049041748 * x1)))) + (-0.29800000786781311 / (1 + math.exp((1.6000000238418579 - 3.7170000076293945 * x0 - 1.3589999675750732 * x1)))) + (-0.19300000369548798 / (1 + math.exp((5.0430002212524414 - 3.7460000514984131 * x0 - 0.57999998331069946 * x1)))) + (0.44200000166893005 / (1 + math.exp((1.4500000476837158 + 2.2709999084472656 * x0 - 0.035999998450279236 * x1)))) + (0.44499999284744263 / (1 + math.exp((2.2880001068115234 - 2.7179999351501465 * x0 + 0.27500000596046448 * x1)))) + (0.87999999523162842 / (1 + math.exp((0.89200001955032349 + 2.6979999542236328 * x0 - 2.875999927520752 * x1)))) + (0.96299999952316284 / (1 + math.exp((4.7290000915527344 - 2.1909999847412109 * x0 + 1.8890000581741333 * x1)))) + (1.4220000505447388 / (1 + math.exp((4.5869998931884766 - 0.14499999582767487 * x0 + 2.1189999580383301 * x1))))), 2) - 0.31099998950958252 * pow((1.6069999933242798 + (-1.8519999980926514 / (1 + math.exp((1.6000000238418579 - 3.7170000076293945 * x0 - 1.3589999675750732 * x1)))) + (-1.3839999437332153 / (1 + math.exp((5.0430002212524414 - 3.7460000514984131 * x0 - 0.57999998331069946 * x1)))) + (-1.0299999713897705 / (1 + math.exp((0.18500000238418579 + 0.59799998998641968 * x0 - 2.3910000324249268 * x1)))) + (-0.49300000071525574 / (1 + math.exp((2.2880001068115234 - 2.7179999351501465 * x0 + 0.27500000596046448 * x1)))) + (-0.29699999094009399 / (1 + math.exp((-1.5820000171661377 - 0.32499998807907104 * x0 - 2.9619998931884766 * x1)))) + (-0.15700000524520874 / (1 + math.exp((4.7290000915527344 - 2.1909999847412109 * x0 + 1.8890000581741333 * x1)))) + (0.17700000107288361 / (1 + math.exp((0.44100001454353333 + 0.86799997091293335 * x0 - 2.3900001049041748 * x1)))) + (0.60100001096725464 / (1 + math.exp((4.5869998931884766 - 0.14499999582767487 * x0 + 2.1189999580383301 * x1)))) + (1.0049999952316284 / (1 + math.exp((0.89200001955032349 + 2.6979999542236328 * x0 - 2.875999927520752 * x1)))) + (1.0800000429153442 / (1 + math.exp((1.4500000476837158 + 2.2709999084472656 * x0 - 0.035999998450279236 * x1))))), 2))
    return barrier
"""
fossil_eval = py"eval_fossil_barrier";

function B_lex(state, zero_level_barriers, one_level_barriers, two_level_barriers)
   b0, u0 = maximum([ (B(state...), u) for (B, u) in zero_level_barriers])
   if (b0 >= 0)
        return b0, u0
    end
   b1, u1 = maximum([ (B(state...), u) for (B, u) in one_level_barriers])
   if (b1 >= 0)
        return b1, u1
    end
   b2, u2 = maximum([ (B(state...), u) for (B, u) in two_level_barriers])
   if (b2 >= 0)
        return b2, u2
   end
   return max((b0, u0), (b1, u1), (b2, u2))
end

function fossil_comparisons(zero_level_barriers,first_level_barriers,second_level_barriers, fossil_eval, test_pts_fixed_seed)
    counter = 0
    counter_f = 0
    counter_s = 0
    counter_x = 0
    for pt in test_pts_fixed_seed
        c,_ = B_lex(pt, zero_level_barriers, first_level_barriers, second_level_barriers )
        if (fossil_eval(pt[1],pt[2]))>=0 && -c < 0
            counter = counter + 1
        end
        if (fossil_eval(pt[1],pt[2]))<0
            counter_f = counter_f + 1
        end
        if (-c)<0
            counter_s = counter_s + 1
        end
        if (fossil_eval(pt[1],pt[2]))<0 && -c >= 0
            counter_x = counter_x + 1
        end
    end
    println("Total test points: 1000")
    println("Safe in FOSSIL: $(counter)")
    println("Safe in Ours: $(counter_s)")
    println("Unsafe in FOSSIL and Safe in Ours: $(counter_f)")
    println("Safe in FOSSIL and Unsafe in Ours: $(counter_x)")
    
    return counter_f, counter_s, counter, counter_x
end

In [ ]:
fossil_comparisons(zero_level_barriers,first_level_barriers,second_level_barriers, fossil_eval, test_pts_fixed_seed)

# 5. Verification

In [ ]:
"""
Certifying barriers using constraint PSD check
"""

using DelimitedFiles

function check_psd_constraints(multipliers_array::Vector)
    counter = 0
    M = []
    for mults in multipliers_array
        for lms in mults
            for lm in lms
                if !isposdef(Matrix(lm.Q)) && !isposdef(Diagonal(svd(Matrix(lm.Q)).S))
                    counter += 1
                else 
#                     display(Matrix(lm.Q))
                    push!(M, [Matrix(lm.Q)])
                end
            end
        end
    end
    open("matrices_p1.txt", "w") do io
        writedlm(io, M)
    end
    if counter==0
        println("All constraints are PSD: Barrier certified")
    else
        println("PSD check failed!")
    end
end

In [ ]:
all_barrier_plus = push!([first_level_barriers], second_level_barriers);

In [ ]:
lms=[]
for i in all_barrier_plus
    if (!ismissing(i[1][4]))
        lms = append!(lms,i[1][4])
    end
end

In [ ]:
check_psd_constraints([lm0,lm1,lms])

In [ ]:
value(all_barrier_plus[1][1][5])

In [ ]:
all_barrier_plus[1][1][5]

In [ ]:
cs=[]
for i in all_barrier_plus
    if (!ismissing(i[1][5]))
        lms = append!(cs,value(i[1][5]))
    end
end

In [ ]:
cs